In [2]:
import pandas as pd
import os

path = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + os.sep + "data" + os.sep + "mbti_preprocessed_complete.csv"
df = pd.read_csv(path, index_col=0)
df.head()

,type,posts,encodedType,preprocessed_posts,extro_intro,intu_obs,feel_think,prosp_judg
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,8,"['youtube', 'tumblr', 'enfp', 'intj', 'moment'...",0,1,1,0
1,ENTP,'I'm finding the lack of me in these posts ver...,3,"['im', 'finding', 'the', 'lack', 'of', 'post',...",1,1,0,1
2,INTP,'Good one _____ https://www.youtube.com/wat...,11,"['good', 'one', 'youtube', 'of', 'course', 'i'...",0,1,0,1
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",10,"['dear', 'intp', 'i', 'enjoyed', 'conversation...",0,1,0,0
4,ENTJ,'You're fired.|||That's another silly misconce...,2,"['youre', 'fired', 'thats', 'another', 'silly'...",1,1,0,0


In [3]:
from sklearn.model_selection import train_test_split

def create_train_test_split(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42069)
    return X_train, X_test, Y_train, Y_test

Extroverted Introverted

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = df.iloc[:, 3].values #Preprocessed_posts
Y = df.iloc[:, 4].values #extro_intro

In [5]:
vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.3)
matrix = vectorizer.fit_transform(X)
print("Vectorizer created {} features.".format(len(vectorizer.get_feature_names_out())))
X = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names_out())
display(X)

X_train, X_test, Y_train, Y_test = create_train_test_split(X, Y)

Vectorizer created 620 features.


,ability,absolutely,accept,accurate,across,act,action,actual,add,admit,...,writing,written,wrote,xd,yesterday,youd,youll,young,younger,youve
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.096755
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.080565,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
2,0.175949,0.152955,0.0,0.092026,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.075340,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
3,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.151928,0.088888,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.074559,0.00000,0.000000,0.132613
4,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.095449,0.0,0.092035,0.0,0.0,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.081479,...,0.000000,0.000000,0.0,0.078123,0.0,0.0,0.068736,0.00000,0.071837,0.000000
8671,0.000000,0.000000,0.0,0.000000,0.134748,0.0,0.000000,0.000000,0.0,0.000000,...,0.287795,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8672,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8673,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.069657,0.0,0.000000,...,0.053322,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000


In [6]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver = "lbfgs", max_iter=3000)
classifier.fit(X_train, Y_train)

LogisticRegression(max_iter=3000)

In [7]:
#predit y
Y_pred = classifier.predict(X_test)

#evaluate  
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(accuracy_score(Y_test, Y_pred))


[[1903   99]
 [ 368  233]]
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      2002
           1       0.70      0.39      0.50       601

    accuracy                           0.82      2603
   macro avg       0.77      0.67      0.70      2603
weighted avg       0.81      0.82      0.80      2603

0.8205916250480215


Intuitive - Observant

In [8]:
X = df.iloc[:, 3].values #Preprocessed_posts
Y = df.iloc[:, 5].values #extro_intro

vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.3)
matrix = vectorizer.fit_transform(X)
print("Vectorizer created {} features.".format(len(vectorizer.get_feature_names_out())))
X = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names_out())
display(X)

X_train, X_test, Y_train, Y_test = create_train_test_split(X, Y)

classifier = LogisticRegression(solver = "lbfgs", max_iter=3000)
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)

print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(accuracy_score(Y_test, Y_pred))




Vectorizer created 620 features.


,ability,absolutely,accept,accurate,across,act,action,actual,add,admit,...,writing,written,wrote,xd,yesterday,youd,youll,young,younger,youve
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.096755
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.080565,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
2,0.175949,0.152955,0.0,0.092026,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.075340,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
3,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.151928,0.088888,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.074559,0.00000,0.000000,0.132613
4,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.095449,0.0,0.092035,0.0,0.0,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.081479,...,0.000000,0.000000,0.0,0.078123,0.0,0.0,0.068736,0.00000,0.071837,0.000000
8671,0.000000,0.000000,0.0,0.000000,0.134748,0.0,0.000000,0.000000,0.0,0.000000,...,0.287795,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8672,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8673,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.069657,0.0,0.000000,...,0.053322,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000


[[  69  273]
 [  30 2231]]
              precision    recall  f1-score   support

           0       0.70      0.20      0.31       342
           1       0.89      0.99      0.94      2261

    accuracy                           0.88      2603
   macro avg       0.79      0.59      0.62      2603
weighted avg       0.87      0.88      0.85      2603

0.8835958509412216


Feeling - Thinking

In [9]:
X = df.iloc[:, 3].values #Preprocessed_posts
Y = df.iloc[:, 5].values #feel_think

vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.3)
matrix = vectorizer.fit_transform(X)
print("Vectorizer created {} features.".format(len(vectorizer.get_feature_names_out())))
X = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names_out())
display(X)

X_train, X_test, Y_train, Y_test = create_train_test_split(X, Y)

classifier = LogisticRegression(solver = "lbfgs", max_iter=3000)
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)

print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(accuracy_score(Y_test, Y_pred))


Vectorizer created 620 features.


,ability,absolutely,accept,accurate,across,act,action,actual,add,admit,...,writing,written,wrote,xd,yesterday,youd,youll,young,younger,youve
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.096755
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.080565,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
2,0.175949,0.152955,0.0,0.092026,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.075340,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
3,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.151928,0.088888,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.074559,0.00000,0.000000,0.132613
4,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.095449,0.0,0.092035,0.0,0.0,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.081479,...,0.000000,0.000000,0.0,0.078123,0.0,0.0,0.068736,0.00000,0.071837,0.000000
8671,0.000000,0.000000,0.0,0.000000,0.134748,0.0,0.000000,0.000000,0.0,0.000000,...,0.287795,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8672,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8673,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.069657,0.0,0.000000,...,0.053322,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000


[[  69  273]
 [  30 2231]]
              precision    recall  f1-score   support

           0       0.70      0.20      0.31       342
           1       0.89      0.99      0.94      2261

    accuracy                           0.88      2603
   macro avg       0.79      0.59      0.62      2603
weighted avg       0.87      0.88      0.85      2603

0.8835958509412216


Prospective - Judging

In [10]:
X = df.iloc[:, 3].values #Preprocessed_posts
Y = df.iloc[:, 6].values #pros_judg

vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.3)
matrix = vectorizer.fit_transform(X)
print("Vectorizer created {} features.".format(len(vectorizer.get_feature_names_out())))
X = pd.DataFrame(matrix.toarray(), columns=vectorizer.get_feature_names_out())
display(X)

X_train, X_test, Y_train, Y_test = create_train_test_split(X, Y)

classifier = LogisticRegression(solver = "lbfgs", max_iter=3000)
classifier.fit(X_train, Y_train)

Y_pred = classifier.predict(X_test)

print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))
print(accuracy_score(Y_test, Y_pred))


Vectorizer created 620 features.


,ability,absolutely,accept,accurate,across,act,action,actual,add,admit,...,writing,written,wrote,xd,yesterday,youd,youll,young,younger,youve
0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.096755
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.080565,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
2,0.175949,0.152955,0.0,0.092026,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.075340,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
3,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.151928,0.088888,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.074559,0.00000,0.000000,0.132613
4,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.095449,0.0,0.092035,0.0,0.0,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8670,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.081479,...,0.000000,0.000000,0.0,0.078123,0.0,0.0,0.068736,0.00000,0.071837,0.000000
8671,0.000000,0.000000,0.0,0.000000,0.134748,0.0,0.000000,0.000000,0.0,0.000000,...,0.287795,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8672,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000
8673,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.069657,0.0,0.000000,...,0.053322,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000


[[ 913  286]
 [ 227 1177]]
              precision    recall  f1-score   support

           0       0.80      0.76      0.78      1199
           1       0.80      0.84      0.82      1404

    accuracy                           0.80      2603
   macro avg       0.80      0.80      0.80      2603
weighted avg       0.80      0.80      0.80      2603

0.8029197080291971
